# `imbrium` use case
## Hyper parameter optimization - Bidirectional Long Short-term Neural Network

### Multivariate pure forecasting

##### Example Steps:

- basic data preparation
- scale target and feature numpy arrays
- create imbrium bidirectional long short-term neural network
- use optuna to perform hyper parameter optimzation on shifting window variables

In [ ]:
import imbrium
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import optuna

print(f"imbrium version: {imbrium.__version__} loaded")

In [ ]:
example_data = pd.read_csv('example_dataset/AirQualityUCI.csv', delimiter=';')
example_data.head()

In [ ]:
example_data = example_data[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [ ]:
example_data.isna().sum()

In [ ]:
example_data = example_data.dropna()

In [ ]:
example_data.notna().sum()

In [ ]:
example_data = example_data.replace(",", ".", regex = True).astype("float")

In [ ]:
target = np.array(example_data['PT08.S1(CO)']).reshape(-1, 1)

In [ ]:
features = example_data[['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [ ]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target)

In [ ]:
custom_layer_config = {
            "layer0": {
                "config": {
                    "neurons": 200,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer1": {
                "config": {
                    "neurons": 100,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer2": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.002
                }
            },
            "layer4": {
                "config": {
                    "neurons": 25,
                    "activation": "relu",
                    "regularization": 0.002,
                }
            },
        }

In [ ]:
from imbrium import PureMulti

In [ ]:
predictor_instance = PureMulti(target = scaled_target, features = scaled_features)


def objective(trial):

    steps_past = trial.suggest_int('steps_past', 5, 30)
    steps_future = trial.suggest_int('steps_future', 5, 10)
    
    
    metric_value = predictor_instance.create_fit_bilstm(
        steps_past =  steps_past,
        steps_future = steps_future,
        loss='mean_squared_error',
        metrics='mean_squared_error',
        bilstm_block_one = 3,
        lstm_block_one = 2,
        layer_config = custom_layer_config, 
        epochs=10,
        show_progress=1,
        validation_split=0.20,
        board=False,
        monitor='val_loss',
        patience=3,
        min_delta=0,
        verbose=1
    )

    return metric_value

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=20) 